|||
|---|---|
|Title|nwtimetracking|
|Author|numbworks|
|Version|3.7.0|
||Please check [docs/docs-nwtimetracking.md](../docs/docs-nwtimetracking.md) before proceeding.|

#### Global Modules

In [1]:
from pandas import DataFrame
from pandas.io.formats.style import Styler

#### Local Modules

In [2]:
from nwtimetracking import SettingBag, DefaultPathProvider, YearProvider, SoftwareProjectNameProvider, TimeTrackingManager
from nwtimetracking import ComponentBag, MarkdownProcessor
from nwshared import VersionChecker, DisplayPreProcessor
from nwpackageversions import StatusChecker

#### Python Version Check

In [3]:
status : str = VersionChecker().get_python_version_status(required = (3, 12, 5))
print(status)

The installed Python version is matching the expected one (installed: '3.12.5', expected: '3.12.5').


#### Dependency Status Check

In [4]:
enable_dependency_status_check : bool = False

if enable_dependency_status_check:
    dockerfile_path : str = os.path.join(os.path.abspath(os.curdir).replace("src", ".devcontainer"), "Dockerfile")
    status_checker : StatusChecker = StatusChecker(logging_function = lambda x : None)
    print(status_checker.try_check(file_path = dockerfile_path))

#### Functions : Temp

#### Settings

In [5]:
setting_bag : SettingBag = SettingBag(
    years = YearProvider().get_all_years(),
    yearly_targets = YearProvider().get_all_yearly_targets(),
    excel_path = DefaultPathProvider().get_default_time_tracking_path(),
    excel_books_nrows = 1242,
    software_project_names = SoftwareProjectNameProvider().get_all_software_project_names(),
    software_project_names_by_spv = SoftwareProjectNameProvider().get_all_software_project_names_by_spv(), 
    tt_by_year_hashtag_years = [2024],
    show_tts_by_month_md = False,
    save_tts_by_month_md = False
)


#### Main : Analysis

In [6]:
tt_manager : TimeTrackingManager = TimeTrackingManager()
sessions_df : DataFrame = tt_manager.get_sessions_dataset(setting_bag = setting_bag)
dpp : DisplayPreProcessor = DisplayPreProcessor()      

if setting_bag.show_sessions_df:
    display(sessions_df.tail(n = setting_bag.n_generic))


In [7]:
if setting_bag.show_tt_by_year_df:
    
    tt_by_year_df : DataFrame = tt_manager.get_tt_by_year(
        sessions_df = sessions_df, 
        years = setting_bag.years, 
        yearly_targets = setting_bag.yearly_targets)
    
    styler : Styler = dpp.hide_index(df = tt_by_year_df)
    display(styler)


Year,Effort,YearlyTarget,TargetDiff,IsTargetMet
2015,18h 00m,00h 00m,+18h 00m,True
2016,615h 15m,500h 00m,+115h 15m,True
2017,762h 45m,500h 00m,+262h 45m,True
2018,829h 45m,500h 00m,+329h 45m,True
2019,515h 15m,500h 00m,+15h 15m,True
2020,470h 30m,500h 00m,-30h 30m,False
2021,537h 30m,500h 00m,+37h 30m,True
2022,467h 30m,400h 00m,+67h 30m,True
2023,320h 15m,250h 00m,+70h 15m,True
2024,549h 15m,500h 00m,+49h 15m,True


In [8]:
if setting_bag.show_tt_by_year_month_df:

    tt_by_year_month_df : DataFrame = tt_manager.get_tt_by_year_month(
        sessions_df = sessions_df, 
        years = setting_bag.years, 
        yearly_targets = setting_bag.yearly_targets)
    
    display(tt_by_year_month_df.tail(n = setting_bag.n_generic))


,Year,Month,Effort,YearlyTotal,ToTarget
103,2024,6,48h 00m,321h 45m,-179h 45m
104,2024,7,67h 00m,388h 45m,-112h 45m
105,2024,8,32h 45m,421h 30m,-79h 30m
106,2024,9,48h 00m,469h 30m,-31h 30m
107,2024,10,79h 45m,549h 15m,+49h 15m


In [9]:
if setting_bag.show_tt_by_year_month_spnv_df:

    tt_by_year_month_spnv_df : DataFrame = tt_manager.get_tt_by_year_month_spnv(
        sessions_df = sessions_df, 
        years = setting_bag.years, 
        software_project_names = setting_bag.software_project_names)
    
    display(tt_by_year_month_spnv_df)

    tt_manager.try_print_definitions(df = tt_by_year_month_spnv_df, definitions = setting_bag.definitions)


In [10]:
if setting_bag.show_tt_by_year_spnv_df:

    tt_by_year_spnv_df : DataFrame = tt_manager.get_tt_by_year_spnv(
        sessions_df = sessions_df, 
        years = setting_bag.years, 
        software_project_names = setting_bag.software_project_names)
    
    display(tt_by_year_spnv_df)
    
    tt_manager.try_print_definitions(df = tt_by_year_spnv_df, definitions = setting_bag.definitions)


In [11]:
if setting_bag.show_tt_by_spn_df:

    tt_by_spn_df : DataFrame = tt_manager.get_tt_by_spn(
        sessions_df = sessions_df, 
        years = setting_bag.years, 
        software_project_names = setting_bag.software_project_names,
        remove_untagged = setting_bag.remove_untagged_from_de
        )
    
    styler : Styler = dpp.hide_index(df = tt_by_spn_df, formatters = { "%_DE" : "{:.2f}", "%_TE" : "{:.2f}" })
    display(styler)
    
    tt_manager.try_print_definitions(df = tt_by_spn_df, definitions = setting_bag.definitions)


Hashtag,ProjectName,Effort,DE,%_DE,TE,%_TE
#python,nwtraderaanalytics,217h 15m,1808h 45m,12.01,2537h 15m,8.56
#python,nwreadinglist,113h 00m,1808h 45m,6.25,2537h 15m,4.45
#python,nwtimetracking,83h 00m,1808h 45m,4.59,2537h 15m,3.27
#python,nwpackageversions,39h 45m,1808h 45m,2.20,2537h 15m,1.57
#python,nwshared,24h 15m,1808h 45m,1.34,2537h 15m,0.96
#csharp,NW.UnivariateForecasting,208h 00m,1808h 45m,11.50,2537h 15m,8.20
#csharp,NW.NGramTextClassification,207h 30m,1808h 45m,11.47,2537h 15m,8.18
#csharp,NW.MarkdownTables,20h 45m,1808h 45m,1.15,2537h 15m,0.82
#csharp,NW.Shared.Files,05h 30m,1808h 45m,0.30,2537h 15m,0.22
#csharp,NW.Shared.Serialization,04h 15m,1808h 45m,0.23,2537h 15m,0.17


DE: Development Effort
TE: Total Effort


In [12]:
if setting_bag.show_tt_by_spn_df:

    tt_by_spn_spv_df : DataFrame = tt_manager.get_tt_by_spn_spv(
        sessions_df = sessions_df, 
        years = setting_bag.years, 
        software_project_names = setting_bag.software_project_names_by_spv)
    
    styler : Styler = dpp.hide_index(df = tt_by_spn_spv_df)
    display(styler)


ProjectName,ProjectVersion,Effort
NW.MarkdownTables,1.0.0,15h 15m
NW.MarkdownTables,1.0.1,02h 30m
NW.MarkdownTables,3.0.0,03h 00m
NW.NGramTextClassification,1.0.0,73h 15m
NW.NGramTextClassification,1.1.0,07h 30m
NW.NGramTextClassification,2.0.0,15h 30m
NW.NGramTextClassification,3.0.0,28h 00m
NW.NGramTextClassification,3.5.0,39h 45m
NW.NGramTextClassification,3.6.0,17h 30m
NW.NGramTextClassification,3.7.0,08h 00m


In [13]:
if setting_bag.show_tt_by_year_hashtag:

    tt_by_year_hashtag_df : DataFrame = tt_manager.get_tt_by_year_hashtag(
        sessions_df = sessions_df, 
        years = setting_bag.tt_by_year_hashtag_years)
    
    styler : Styler = dpp.hide_index(df = tt_by_year_hashtag_df)
    display(styler)


Year,Hashtag,Effort
2024,#csharp,35h 15m
2024,#maintenance,102h 00m
2024,#python,333h 00m
2024,#studying,79h 00m


In [14]:
if setting_bag.show_tt_by_hashtag:
    tt_by_hashtag_df : DataFrame = tt_manager.get_tt_by_hashtag(sessions_df = sessions_df)
    styler : Styler = dpp.hide_index(df = tt_by_hashtag_df, formatters = { "Effort%" : "{:.2f}" })
    display(styler)


Hashtag,Effort,Effort%
#untagged,2548h 45m,50.11
#csharp,1116h 45m,21.96
#python,546h 00m,10.74
#studying,373h 45m,7.35
#maintenance,327h 45m,6.44
#powershell,154h 00m,3.03
#overtime,19h 00m,0.37


In [15]:
if setting_bag.show_tts_by_month_df:
    
    tts_by_month_df : DataFrame = tt_manager.get_tts_by_month(sessions_df = sessions_df, years = setting_bag.years)
    tts_by_month_upd_df : DataFrame = tt_manager.update_future_months_to_empty(tts_by_month_df = tts_by_month_df, now = setting_bag.now)

    if setting_bag.tts_by_month_update_future_values_to_empty:
        styler : Styler = dpp.hide_index(df = tts_by_month_upd_df)
        display(styler)
    else:
        styler : Styler = dpp.hide_index(df = tts_by_month_df)
        display(styler)


Month,2015,↕,2016,↕,2017,↕,2018,↕,2019,↕,2020,↕,2021,↕,2022,↕,2023,↕,2024
1,00h 00m,↑,18h 00m,↑,88h 30m,↓,80h 15m,↓,60h 00m,↓,29h 15m,↑,53h 00m,↓,00h 00m,↑,06h 00m,↑,45h 45m
2,00h 00m,↑,45h 30m,↑,65h 30m,↑,90h 45m,↓,73h 00m,↓,38h 00m,↓,31h 30m,↓,03h 00m,↑,24h 00m,↑,77h 45m
3,00h 00m,↑,20h 45m,↑,71h 45m,↑,89h 00m,↓,75h 30m,↓,35h 00m,↑,40h 30m,↓,06h 15m,↑,50h 15m,↑,77h 45m
4,00h 00m,↑,37h 30m,↑,68h 00m,↑,88h 30m,↓,59h 45m,↓,40h 45m,↓,19h 00m,↑,27h 30m,↓,19h 00m,↑,29h 30m
5,00h 00m,↑,53h 00m,↑,83h 00m,↑,91h 15m,↓,54h 45m,↓,14h 30m,↑,112h 45m,↓,49h 45m,↓,31h 00m,↑,43h 00m
6,00h 00m,↑,57h 45m,↓,37h 45m,↑,62h 00m,↓,29h 15m,↓,12h 00m,↑,54h 00m,↑,73h 30m,↓,24h 45m,↑,48h 00m
7,00h 00m,↑,46h 45m,↑,65h 30m,↑,69h 30m,↓,24h 15m,↑,34h 00m,↓,23h 30m,↑,51h 00m,↓,16h 30m,↑,67h 00m
8,00h 00m,↑,25h 45m,↑,45h 45m,↑,72h 00m,↓,06h 00m,↑,32h 00m,↑,110h 00m,↓,36h 30m,↑,41h 30m,↓,32h 45m
9,00h 00m,↑,89h 30m,↓,43h 45m,↑,64h 00m,↓,39h 00m,↑,44h 00m,↓,43h 30m,↑,69h 00m,↓,50h 15m,↓,48h 00m
10,08h 00m,↑,82h 15m,↓,64h 30m,↓,46h 45m,↓,45h 30m,↑,48h 00m,↓,35h 30m,↑,38h 30m,↓,20h 00m,↑,79h 45m


In [16]:
if (setting_bag.show_effort_status_df):
    
    es_df : DataFrame = tt_manager.add_effort_status(sessions_df = sessions_df)
    es_df = tt_manager.filter_by_is_correct(es_df = es_df, is_correct = setting_bag.effort_status_is_correct)
    
    if es_df.empty:
        styler : Styler = dpp.hide_index(df = es_df)
        display(styler)
    else:
        styler : Styler = dpp.hide_index(df = es_df.head(n = setting_bag.effort_status_n))
        display(styler)


StartTime,EndTime,Effort,ES_IsCorrect,ES_Expected,ES_Message
22:15,23:00,1h 00m,False,00h 45m,"The provided row contains a mismatching effort (idx: '1234', start_time: '22:15', end_time: '23:00', actual_effort: '1h 00m', expected_effort: '00h 45m')."


In [17]:
if setting_bag.show_time_ranges_df:

    time_ranges_df : DataFrame = tt_manager.create_time_ranges_df(
        sessions_df = sessions_df, 
        unknown_id = setting_bag.time_ranges_unknown_id)
    
    if setting_bag.show_time_ranges_df:
        time_ranges_df = tt_manager.remove_unknown_id(
            time_ranges_df = time_ranges_df, 
            unknown_id = setting_bag.time_ranges_unknown_id)
        
    if setting_bag.time_ranges_filter_by_top_n:
        time_ranges_df = tt_manager.filter_by_top_n_occurrences(
            time_ranges_df = time_ranges_df, 
            n = setting_bag.time_ranges_top_n)

    styler : Styler = dpp.hide_index(df = time_ranges_df)
    display(styler)


TimeRangeId,Occurrences
08:00-08:45,37
08:00-08:30,25
18:00-20:00,21
19:00-20:00,17
17:30-18:00,17


#### Main : Markdown

In [18]:
component_bag : ComponentBag = ComponentBag()
markdown_processor : MarkdownProcessor = MarkdownProcessor(component_bag = component_bag, setting_bag = setting_bag)

markdown_processor.process_tts_by_month_md(tts_by_month_upd_df = tts_by_month_upd_df)